In [48]:
%cd ../decoding

/Users/gilad/Desktop/Projects/Semantic Decoding/semantic-decoding/decoding


In [49]:
import os
from os.path import join, dirname
import numpy as np
import json
import argparse
import h5py
import numpy as np
import torch
import scipy.stats as ss


In [50]:
import config
from GPT import GPT
from utils_stim import get_stim
from utils_resp import get_resp
from utils_ridge.ridge import ridge, bootstrap_ridge
from utils_ridge.textgrid import TextGrid


In [51]:
np.random.seed(42)


In [52]:
REPO_DIR = os.getcwd()
DATA_LM_DIR = os.path.join(REPO_DIR, "data_lm")
DATA_TRAIN_DIR = os.path.join(REPO_DIR, "data_train")
DATA_TEST_DIR = os.path.join(REPO_DIR, "data_test")
MODEL_DIR = os.path.join(REPO_DIR, "models")
RESULT_DIR = os.path.join(REPO_DIR, "results")
SCORE_DIR = os.path.join(REPO_DIR, "scores")

# GPT encoding model parameters

TRIM = 5
STIM_DELAYS = [1, 2, 3, 4]
RESP_DELAYS = [-4, -3, -2, -1]
ALPHAS = np.logspace(1, 3, 10)
NBOOTS = 50
VOXELS = 10000
CHUNKLEN = 40
GPT_LAYER = 9
GPT_WORDS = 5

# decoder parameters

RANKED = True
WIDTH = 200
NM_ALPHA = 2/3
LM_TIME = 8
LM_MASS = 0.9
LM_RATIO = 0.1
EXTENSIONS = 5

# evaluation parameters

WINDOW = 20

# devices

GPT_DEVICE = "cpu"
EM_DEVICE = "cpu"
SM_DEVICE = "cpu"

In [53]:
#import config
from GPT import GPT


## Encoder Training Parameters

In [54]:
subject = "UTS01"
gpt = "perceived"
sessions = "9"

In [55]:
REPO_DIR = os.path.dirname(os.getcwd())

In [56]:
DATA_LM_DIR = os.path.join(REPO_DIR, "data_lm")
DATA_TRAIN_DIR = os.path.join(REPO_DIR, "data_train")
DATA_TEST_DIR = os.path.join(REPO_DIR, "data_test")
MODEL_DIR = os.path.join(REPO_DIR, "models")
RESULT_DIR = os.path.join(REPO_DIR, "results")
SCORE_DIR = os.path.join(REPO_DIR, "scores")

## Loading Stores

In [57]:
class LMFeatures():
    """class for extracting contextualized features of stimulus words
    """
    def __init__(self, model, layer, context_words):
        self.model, self.layer, self.context_words = model, layer, context_words

    def extend(self, extensions, verbose = False):
        """outputs array of vectors corresponding to the last words of each extension
        """
        contexts = [extension[-(self.context_words+1):] for extension in extensions]
        if verbose: print(contexts)
        context_array = self.model.get_context_array(contexts)
        embs = self.model.get_hidden(context_array, layer = self.layer)
        return embs[:, len(contexts[0]) - 1]

    def make_stim(self, words):
        """outputs matrix of features corresponding to the stimulus words
        """
        context_array = self.model.get_story_array(words, self.context_words)
        embs = self.model.get_hidden(context_array, layer = self.layer)
        return np.vstack([embs[0, :self.context_words], 
            embs[:context_array.shape[0] - self.context_words, self.context_words]])

In [58]:
stories = []
with open(os.path.join(DATA_TRAIN_DIR, "sess_to_story.json"), "r") as f:
    sess_to_story = json.load(f) 
for sess in sessions:
    stories.extend(sess_to_story[str(sess)])


In [59]:
with open(os.path.join(DATA_LM_DIR, gpt, "vocab.json"), "r") as f:
    gpt_vocab = json.load(f)
gpt = GPT(path = os.path.join(DATA_LM_DIR, gpt, "model"), vocab = gpt_vocab, device = config.GPT_DEVICE)
features = LMFeatures(model = gpt, layer = config.GPT_LAYER, context_words = config.GPT_WORDS)


## Train Encoding model

In [60]:
def get_stim(stories, features, tr_stats = None):
    """extract quantitative features of stimulus stories
    """
    word_seqs = get_story_wordseqs(stories)
    word_vecs = {story : features.make_stim(word_seqs[story].data) for story in stories}
    word_mat = np.vstack([word_vecs[story] for story in stories])
    word_mean, word_std = word_mat.mean(0), word_mat.std(0)
    
    ds_vecs = {story : lanczosinterp2D(word_vecs[story], word_seqs[story].data_times, word_seqs[story].tr_times) 
               for story in stories}
    ds_mat = np.vstack([ds_vecs[story][5+TRIM:-TRIM] for story in stories])
    if tr_stats is None: 
        r_mean, r_std = ds_mat.mean(0), ds_mat.std(0)
        r_std[r_std == 0] = 1
    else: 
        r_mean, r_std = tr_stats
    ds_mat = np.nan_to_num(np.dot((ds_mat - r_mean), np.linalg.inv(np.diag(r_std))))
    del_mat = make_delayed(ds_mat, STIM_DELAYS)
    if tr_stats is None: return del_mat, (r_mean, r_std), (word_mean, word_std)
    else: return del_mat

In [61]:
def get_resp(subject, stories, stack = True, vox = None):
    """loads response data
    """
    subject_dir = os.path.join(config.DATA_TRAIN_DIR, "train_response", subject)
    resp = {}
    for story in stories:
        resp_path = os.path.join(subject_dir, "%s.hf5" % story)
        hf = h5py.File(resp_path, "r")
        resp[story] = np.nan_to_num(hf["data"][:])
        if vox is not None:
            resp[story] = resp[story][:, vox]
        hf.close()
    if stack: return np.vstack([resp[story] for story in stories]) 
    else: return resp    


In [41]:

def predict_word_times(word_rate, resp, starttime = 0, tr = 2):
    """predict evenly spaced word times from word rate
    """
    half = tr / 2
    trf = TRFile(None, tr)
    trf.soundstarttime = starttime
    trf.simulate(resp.shape[0])
    tr_times = trf.get_reltriggertimes() + half

    word_times = []
    for mid, num in zip(tr_times, word_rate):  
        if num < 1: continue
        word_times.extend(np.linspace(mid - half, mid + half, num, endpoint = False) + half / num)
    return np.array(word_times), tr_times

In [42]:
def get_story_wordseqs(stories):
    """loads words and word times of stimulus stories
    """
    grids = load_textgrids(stories, DATA_TRAIN_DIR)
    with open(os.path.join(DATA_TRAIN_DIR, "respdict.json"), "r") as f:
        respdict = json.load(f)
    trfiles = load_simulated_trfiles(respdict)
    wordseqs = make_word_ds(grids, trfiles)
    return wordseqs


In [43]:
def predict_word_rate(resp, wt, vox, mean_rate):
    """predict word rate at each acquisition time
    """
    delresp = make_delayed(resp[:, vox], RESP_DELAYS)
    rate = ((delresp.dot(wt) + mean_rate)).reshape(-1).clip(min = 0)
    return np.round(rate).astype(int)


In [44]:
def load_textgrids(stories, data_dir: str):
    base = join(data_dir, "train_stimulus")
    grids = {}
    for story in stories:
        grid_path = os.path.join(base, "%s.TextGrid" % story)
        grids[story] = TextGrid(open(grid_path).read())
    return grids


In [45]:
def load_simulated_trfiles(respdict, tr=2.0, start_time=10.0, pad=5):
    trdict = dict()
    for story, resps in respdict.items():
        trf = TRFile(None, tr)
        trf.soundstarttime = start_time
        trf.simulate(resps - pad)
        trdict[story] = [trf]
    return trdict

In [46]:
config.DATA_TRAIN_DIR = "/Users/gilad/Desktop/Projects/Semantic Decoding/semantic-decoding/data_train"

### Get the words embedding (stimulus)

In [47]:
rstim, tr_stats, word_stats = get_stim(stories, features)

NameError: name 'make_word_ds' is not defined

In [ ]:
rstim.shape

### Get the response (fMRI)

In [ ]:
rresp = get_resp(subject, stories, stack = True)


In [21]:
rresp.shape

NameError: name 'rresp' is not defined

### ???

In [67]:
nchunks = int(np.ceil(rresp.shape[0] / 5 / config.CHUNKLEN))

### Estimate the ridge model parameters

In [68]:
weights, alphas, bscorrs = bootstrap_ridge(rstim, rresp, use_corr = False, alphas = config.ALPHAS,
    nboots = config.NBOOTS, chunklen = config.CHUNKLEN, nchunks = nchunks)        


In [76]:
weights.shape

(3072, 81126)

In [74]:
bscorrs.shape

(10, 81126, 50)

In [77]:
bscorrs = bscorrs.mean(2).max(0)


In [78]:
bscorrs.shape

(81126,)

### find the best correlated voxels

In [79]:
vox = np.sort(np.argsort(bscorrs)[-config.VOXELS:])

In [82]:
del rstim, rresp

### Estimate noise model

In [84]:
stim_dict = {story : get_stim([story], features, tr_stats = tr_stats) for story in stories}
resp_dict = get_resp(subject, stories, stack = False, vox = vox)
noise_model = np.zeros([len(vox), len(vox)])


In [99]:
for hstory in stories:
    tstim, hstim = np.vstack([stim_dict[tstory] for tstory in stories if tstory != hstory]), stim_dict[hstory]
    tresp, hresp = np.vstack([resp_dict[tstory] for tstory in stories if tstory != hstory]), resp_dict[hstory]
    # use the previously calculated alphas for each voxel, now to estimate the correlation between stimuli and response
    bs_weights = ridge(tstim, tresp, alphas[vox])
    # find the residuals (the parts that are not explained by the model)
    resids = hresp - hstim.dot(bs_weights)
    
    # the noise model which is the amount of noise per story (normalized)
    bs_noise_model = resids.T.dot(resids)
    noise_model += bs_noise_model / np.diag(bs_noise_model).mean() / len(stories)

In [100]:
del stim_dict, resp_dict

### Save the model

In [107]:
gpt

In [108]:
save_location = os.path.join(config.MODEL_DIR, subject)
os.makedirs(save_location, exist_ok = True)
np.savez(os.path.join(save_location, "encoding_model_%s" % "percieved"),
weights = weights, noise_model = noise_model, alphas = alphas, voxels = vox, stories = stories,
tr_stats = np.array(tr_stats), word_stats = np.array(word_stats))

In [109]:
save_location

'/Users/gilad/Desktop/Projects/Semantic Decoding/semantic-decoding/decoding/models/UTS01'

In [110]:
%cd /Users/gilad/Desktop/Projects/Semantic Decoding/semantic-decoding/decoding/models/UTS01

/Users/gilad/Desktop/Projects/Semantic Decoding/semantic-decoding/decoding/models/UTS01


In [111]:
%ls

encoding_model_<GPT.GPT object at 0x7fac00171ac0>.npz
encoding_model_percieved.npz


In [112]:
%ls -l

total 10939184
-rw-r--r--  1 gilad  staff  2794502468 Sep 21 17:56 encoding_model_<GPT.GPT object at 0x7fac00171ac0>.npz
-rw-r--r--  1 gilad  staff  2794502468 Sep 21 18:00 encoding_model_percieved.npz
